In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine
import psycopg2
from config import user, password, location, port, db
from sklearn.metrics import balanced_accuracy_score, accuracy_score, confusion_matrix
import numpy as np

In [2]:
#next 5 cells are the rough plan to get data out of DB and into python
#connection = psycopg2.connect(dbname=db, user=user, password=password, host=location, port=port)

In [3]:
#my_cursor = connection.cursor()

In [4]:
#my_cursor.execute('SELECT t.abbreviation AS 'Home_Team',g.FGM_HOME,g.FG_PCT_HOME,g.FG3_PCT_HOME,g.DREB_HOME,g.AST_HOME,g.FGM_AWAY,g.FG_PCT_AWAY,g.FG3_PCT_AWAY,g.DREB_AWAY,g.LARGEST_LEAD_HOME,g.LARGEST_LEAD_AWAY,g.HOME_TEAM_WINS,g.HOME_TEAM_LOSSES,g.HOME_W FROM game AS g INNER JOIN team AS t ON g.TEAM_ID_HOME=t.id')

In [5]:
# my_result = my_cursor.fetchall()

In [6]:
# columns = ['Home_Team','FGM_HOME','FG_PCT_HOME','FG3_PCT_HOME','DREB_HOME','AST_HOME','FGM_AWAY','FG_PCT_AWAY','FG3_PCT_AWAY','DREB_AWAY','LARGEST_LEAD_HOME','LARGEST_LEAD_AWAY','HOME_TEAM_WINS','HOME_TEAM_LOSSES','HOME_W']
# df = pd.DataFrame(myresult, columns)
# df.head()

In [21]:
#  Import and read the nba data.
file_name = 'Resources/game_edited_corr.csv'
df = pd.read_csv(file_name)
df.head()

,TEAM_ABBREVIATION_HOME,FGM_HOME,FG_PCT_HOME,FG3_PCT_HOME,DREB_HOME,AST_HOME,FGM_AWAY,FG_PCT_AWAY,FG3_PCT_AWAY,DREB_AWAY,LARGEST_LEAD_HOME,LARGEST_LEAD_AWAY,HOME_TEAM_WINS,HOME_TEAM_LOSSES,HOME_W
0,GSW,41,0.427,0.300,35,29,35,0.422,0.333,25,23.0,2.0,3.0,0.0,1
1,CHI,37,0.425,0.368,40,13,38,0.404,0.310,39,13.0,6.0,3.0,1.0,1
2,ATL,37,0.451,0.296,33,22,37,0.385,0.414,36,5.0,19.0,3.0,1.0,0
3,LAL,35,0.389,0.265,36,18,38,0.458,0.250,33,16.0,9.0,1.0,2.0,0
4,PHX,34,0.391,0.250,40,15,40,0.471,0.476,42,5.0,28.0,0.0,3.0,0


In [25]:
#encode team abbr
le = LabelEncoder()
# df['Home_Team'] = le.fit_transform(df['Home_Team'])
# df['TEAM_ID_AWAY'] = le.fit_transform(df['TEAM_ID_AWAY'])
df['TEAM_ABBREVIATION_HOME'] = le.fit_transform(df['TEAM_ABBREVIATION_HOME'])
#df['TEAM_ABBREVIATION_AWAY'] = le.fit_transform(df['TEAM_ABBREVIATION_AWAY'])
df.head()

C:\Users\taylo\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,TEAM_ABBREVIATION_HOME,FGM_HOME,FG_PCT_HOME,FG3_PCT_HOME,DREB_HOME,AST_HOME,FGM_AWAY,FG_PCT_AWAY,FG3_PCT_AWAY,DREB_AWAY,LARGEST_LEAD_HOME,LARGEST_LEAD_AWAY,HOME_TEAM_WINS,HOME_TEAM_LOSSES,HOME_W
0,9,41,0.427,0.300,35,29,35,0.422,0.333,25,23.0,2.0,3.0,0.0,1
1,4,37,0.425,0.368,40,13,38,0.404,0.310,39,13.0,6.0,3.0,1.0,1
2,0,37,0.451,0.296,33,22,37,0.385,0.414,36,5.0,19.0,3.0,1.0,0
3,13,35,0.389,0.265,36,18,38,0.458,0.250,33,16.0,9.0,1.0,2.0,0
4,23,34,0.391,0.250,40,15,40,0.471,0.476,42,5.0,28.0,0.0,3.0,0


In [26]:
#drop non-numerical data
df = df.dropna()
drop_df = df.drop(columns=['HOME_W'], axis=1)
drop_df.head()

,TEAM_ABBREVIATION_HOME,FGM_HOME,FG_PCT_HOME,FG3_PCT_HOME,DREB_HOME,AST_HOME,FGM_AWAY,FG_PCT_AWAY,FG3_PCT_AWAY,DREB_AWAY,LARGEST_LEAD_HOME,LARGEST_LEAD_AWAY,HOME_TEAM_WINS,HOME_TEAM_LOSSES
0,9,41,0.427,0.300,35,29,35,0.422,0.333,25,23.0,2.0,3.0,0.0
1,4,37,0.425,0.368,40,13,38,0.404,0.310,39,13.0,6.0,3.0,1.0
2,0,37,0.451,0.296,33,22,37,0.385,0.414,36,5.0,19.0,3.0,1.0
3,13,35,0.389,0.265,36,18,38,0.458,0.250,33,16.0,9.0,1.0,2.0
4,23,34,0.391,0.250,40,15,40,0.471,0.476,42,5.0,28.0,0.0,3.0


In [27]:
# split into features and target arrays
y = df['HOME_W'].values
X = drop_df.values
print(y)

[1 1 0 ... 1 0 1]


In [28]:
# create standard scalar instance
scaler = StandardScaler()

In [29]:
#split preprocesed data into training/testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 10)

In [30]:
#fit the standardscaler
X_scaler = scaler.fit(X_train)

In [31]:
#scaling the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [32]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(solver='lbfgs', random_state=1)

In [33]:
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
   intercept_scaling=1, max_iter=100, multi_class='warn', penalty='12',
   random_state=1, solver='lbfgs', tol=0.0001, warm_start=False)

LogisticRegression(multi_class='warn', penalty='12', random_state=1)

In [34]:
classifier.fit(X_train, y_train)

C:\Users\taylo\anaconda3\envs\mlenv\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(random_state=1)

In [35]:
# generate predictions for test set
y_pred = classifier.predict(X_test)
pd.DataFrame({"Prediction": y_pred, "Actual": y_test})

,Prediction,Actual
0,0,0
1,0,0
2,1,1
3,1,1
4,1,1
...,...,...
1225,1,1
1226,0,0
1227,1,0
1228,1,1


In [36]:
# generate accuracy score
accuracy_score(y_test, y_pred)

0.9203252032520325

In [37]:
cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(cm, index=['Actual Home Loss','Actual Home Win'], columns=['Predicted Home Loss','Predicted Home Win'])
cm_df

,Predicted Home Loss,Predicted Home Win
Actual Home Loss,472,56
Actual Home Win,42,660
